In [26]:
import numpy as np
import pandas as pd
df=pd.read_csv('top10K-TMDB-movies.csv')
print(df)

          id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                 genre original_language  \
0                                          Drama,Crime                en   
1                                 Comedy,Drama,Romance                hi   
2              

In [27]:
a=df.isnull().sum()
print(a)
b=df.describe()
print(b)
c=df.info()
print(c)

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64
                  id    popularity  vote_average    vote_count
count   10000.000000  10000.000000  10000.000000  10000.000000
mean   161243.505000     34.697267      6.621150   1547.309400
std    211422.046043    211.684175      0.766231   2648.295789
min         5.000000      0.600000      4.600000    200.000000
25%     10127.750000      9.154750      6.100000    315.000000
50%     30002.500000     13.637500      6.600000    583.500000
75%    310133.500000     25.651250      7.200000   1460.000000
max    934761.000000  10436.917000      8.700000  31917.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 

In [28]:
#Feature selection part

In [29]:
d=df.columns
print(d)

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')


In [30]:
e=df[['id','title','genre','overview']]

In [31]:
e['tags']=e['genre']+e['overview']
print(e)

          id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                 genre  \
0                                          Drama,Crime   
1                                 Comedy,Drama,Romance   
2                                          Drama,Crime   
3          

<ipython-input-31-75dbb42d13ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e['tags']=e['genre']+e['overview']


In [32]:
movies=e.drop(columns=['overview','genre'])
print(movies)

          id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                   tags  
0     Drama,CrimeFramed in the 1940s for the double ...  
1     Comedy,Drama,RomanceRaj is a rich, carefree, h...  
2     Drama,CrimeSpanning the years 1945 to 1955, a ...  
3     Drama

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=10000,stop_words='english')
x=cv.fit_transform(movies['tags'].values.astype('U')).toarray()
print(x)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
sim=cosine_similarity(x)
print(sim)

[[1.         0.06253054 0.05802589 ... 0.07963978 0.07597372 0.03798686]
 [0.06253054 1.         0.08980265 ... 0.         0.         0.        ]
 [0.05802589 0.08980265 1.         ... 0.02541643 0.03636965 0.        ]
 ...
 [0.07963978 0.         0.02541643 ... 1.         0.03327792 0.03327792]
 [0.07597372 0.         0.03636965 ... 0.03327792 1.         0.04761905]
 [0.03798686 0.         0.         ... 0.03327792 0.04761905 1.        ]]


In [ ]:
def recommand(df):
    index=movies[movies['title']==df].index[0]
    main=sorted(list(enumerate(sim[index])),reverse=True,key=lambda vector:vector[1])
    for i in main[0:5]:
        print(movies.iloc[i[0]].title)

In [61]:
recommand('Dilwale Dulhania Le Jayenge')

Dilwale Dulhania Le Jayenge
The Cheetah Girls: One World
A Passage to India
Fatal Attraction
Noelle


In [66]:
import pickle
pickle.dump(movies,open('df_list.pkl','wb'))
pickle.dump(sim,open('sim_list.pkl','wb'))
p=pickle.load(open('df_list.pkl','rb'))
print(p)

          id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                   tags  
0     Drama,CrimeFramed in the 1940s for the double ...  
1     Comedy,Drama,RomanceRaj is a rich, carefree, h...  
2     Drama,CrimeSpanning the years 1945 to 1955, a ...  
3     Drama